### **Thông Tin Bài Nộp:**
- Nhóm: *04*
- Môn học: *BIG DATA AND ITS APPLICATIONS*
- Giảng viên: *TS. Nguyễn Thôn Dã*
- Bài tập: *PySpark Practice #5*

| StudentID      | Name | Class     |
| :---        |    :---:   |          :--- |
| K204110855 | Đinh Văn An | K20411 |
| K204110568 | Nguyễn Gia Hưng | K20411 |
| K204110579 | Phạm Nguyễn Hiền Phương | K20411 |
| K204111772 | Đinh Hoàn Hảo | K20411 |
| K204110567 | Phạm Thị Minh Hòa | K20411 |
| K194081062 | Lê Thanh Hải | K19408T |

## **Import Libraries**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('PySpark').getOrCreate()

## **4.1. TRANSFORM VALUES IN A COLUMN OF A DATAFRAME**

### **4.1.1. Tạo DataFrame**

Mô tả Credit card fraud Dataset

| Field      | Description | Data Type |
| :---       |  :---       | :---      |
|transdatetrans_time|	Transaction DateTime | DateTime|
|merchant|	Merchant Name |    String|
|category|	Category of Merchant | String|
|amt|	Amount of Transaction | Float|
|city|	City of Credit Card Holder | String|
|state|	State of Credit Card Holder | String|
|lat|	Latitude Location of Purchase | Float|
|long|	Longitude Location of Purchase | Float|
|city_pop|	Credit Card Holder's City Population | Integer|
|job|	Job of Credit Card Holder | String|
|dob|	Date of Birth of Credit Card Holder | DateTime|
|trans_num|	Transaction Number | String|
|merch_lat|	Latitude Location of Merchant | Float|
|merch_long|	Longitude Location of Merchant | Float|
|is_fraud|	Whether Transaction is Fraud (1) or Not (0) | Integer|

Thực hiện load dữ liệu từ file `credit-card-fraud.csv` từ `Github` vào dataframe `is_frauddf` và kiểm tra kiểu dữ liệu của các cột trong dataframe.

In [3]:
# file_location="Data\credit-card-fraud.csv"
# file_type="csv"
# infer_schema="False" 
# first_row_is_header="True"
# df_Origin = spark.read.format(file_type).option("inferSchema", infer_schema).option("header", first_row_is_header).load(file_location)

pandas_df = pd.read_csv('https://raw.githubusercontent.com/thanhENC/Data-set/main/credit-card-fraud.csv')
origin_df = spark.createDataFrame(pandas_df)
origin_df.show(5)
origin_df.printSchema()

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac...|43.150704|-1

**Casting**

Chuyển kiểu dữ liệu cho các cột về đúng kiểu dữ liệu đã được mô tả ở trên ***để tiến hành tính tuổi*** cho người dùng và tạo thành một cột dữ liệu mới.

In [4]:
is_frauddf = origin_df

#identifying and assigning the correct data type to each column
float_vars = ['amt', 'lat', 'long', 'merch_lat', 'merch_long']
date_vars = ['dob']
datetime_vars = ['trans_date_trans_time']
int_vars = ['city_pop', 'is_fraud']
for var in float_vars:
    is_frauddf = is_frauddf.withColumn(var, is_frauddf[var].cast(DoubleType()))
for var in date_vars:
    is_frauddf = is_frauddf.withColumn(var, is_frauddf[var].cast(DateType()))
for var in datetime_vars:
    is_frauddf = is_frauddf.withColumn(var, is_frauddf[var].cast(TimestampType()))
for var in int_vars:
    is_frauddf = is_frauddf.withColumn(var, is_frauddf[var].cast(IntegerType()))

is_frauddf.printSchema()

root
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



In [53]:
is_frauddf.count()

1333

Như vậy ta đã có dataframe `is_frauddf` với các cột đã được chuyển đổi kiểu dữ liệu và có tất cả 1333 dòng dữ liệu.

### **4.1.2. Tính Tuổi Của Card Holder và Thêm Thành Một Cột Dữ Liệu**

In [54]:
is_frauddf = is_frauddf.withColumn('Age', datediff(lit(current_date()), is_frauddf['dob'])/lit(365))
is_frauddf.show(5)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+------------------+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|               Age|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+------------------+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44.33698630136986|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.26

### **4.1.3. Làm Tròn Giá Trị Của Cột Dữ Liệu Tuổi `Age`**

In [55]:
is_frauddf = is_frauddf.withColumn('Age', floor(is_frauddf['Age']))
is_frauddf.show(5)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac

## **4.2. CHỌN CỘT DỮ LIỆU TỪ DATAFRAME**

## **4.5. TẠO VÀ SỬ DỤNG một PYSPARK SQL UDF**

In [78]:
tempDf = is_frauddf
tempDf.show(5)

+---------------------+------------+-------------+-------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|    merchant|     category|    amt|     city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+------------+-------------+-------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 08:19:46|   Berge LLC|gas_transport|   63.6|   Camden|   MO|39.2048| -94.0259|     464| Colour technologist|1959-05-28|be8494e402d166aad...|39.026626| -94.047307|       0| 63|
|  2019-01-01 07:09:10|  Funk Group|  grocery_net|  51.28|  Phoenix|   AZ|33.8155|-112.1202| 1312922|Counselling psych...|1999-11-30|c56c309fa13d1c9ee...|34.166811|-111.595599|       0| 22|
|  2019-01-01 02:54:10|Kassulke PLC| shopping_net|

**Tạo UDF**

In [98]:
def getBirthYear(x):
    return int(str(x).split('-')[0])

In [99]:
getBirthYearUDF = udf(getBirthYear, IntegerType())

In [100]:
tempDfBirthYear = tempDf.withColumn('Year', getBirthYearUDF(tempDf.dob))
tempDfBirthYear.show(5)

+---------------------+------------+-------------+-------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+----+
|trans_date_trans_time|    merchant|     category|    amt|     city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|Year|
+---------------------+------------+-------------+-------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+----+
|  2019-01-01 08:19:46|   Berge LLC|gas_transport|   63.6|   Camden|   MO|39.2048| -94.0259|     464| Colour technologist|1959-05-28|be8494e402d166aad...|39.026626| -94.047307|       0| 63|1959|
|  2019-01-01 07:09:10|  Funk Group|  grocery_net|  51.28|  Phoenix|   AZ|33.8155|-112.1202| 1312922|Counselling psych...|1999-11-30|c56c309fa13d1c9ee...|34.166811|-111.595599|       0| 22|1999|
|  2019-01-01 02:54:10|Ka

**Lưu kết quả vào file csv**

In [109]:
#convert pyspark dataframe to pandas dataframe to save it to csv
tempDfBirthYear.toPandas().to_csv('tempBirthYear.csv', index=False)

## **4.6. GÁN NHÃN DỮ LIỆU**

In [110]:
def labelBirthYear(year):
    if year >= 2010:
        return 'Gen Alpha'
    elif year >= 1997:
        return 'Gen Z'
    elif year >= 1980:
        return 'Millenials'
    elif year >= 1965:
        return 'Gen X'
    elif year >= 1945:
        return 'Baby Boomers'
    else:
        return 'Silent Generation'

In [111]:
labelBirthYearUDF = udf(labelBirthYear, StringType())

In [112]:
tempDfBirthYear2 = tempDfBirthYear.withColumn('Generation', labelBirthYearUDF(tempDfBirthYear.Year))
tempDfBirthYear2.show(5)

+---------------------+------------+-------------+-------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+----+------------+
|trans_date_trans_time|    merchant|     category|    amt|     city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|Year|  Generation|
+---------------------+------------+-------------+-------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+----+------------+
|  2019-01-01 08:19:46|   Berge LLC|gas_transport|   63.6|   Camden|   MO|39.2048| -94.0259|     464| Colour technologist|1959-05-28|be8494e402d166aad...|39.026626| -94.047307|       0| 63|1959|Baby Boomers|
|  2019-01-01 07:09:10|  Funk Group|  grocery_net|  51.28|  Phoenix|   AZ|33.8155|-112.1202| 1312922|Counselling psych...|1999-11-30|c56c309fa13d1c9ee...|34.166811|-111

## **4.7. Thống Kê Mô Tả**

**Sử dụng Pyspark để thực hiện một số mô tả với bộ dữ liệu hiện có**

*Đếm số merchant*

In [56]:
countMer = is_frauddf.agg({'merchant':'count'})
countMer.show(5)

+---------------+
|count(merchant)|
+---------------+
|           1333|
+---------------+



*Tính số lược giao dịch trung bình*

In [57]:
meanAmt = is_frauddf.agg({'amt':'avg'})
meanAmt.show()

+-----------------+
|         avg(amt)|
+-----------------+
|69.16227306826708|
+-----------------+



*Tính phương sai của cột giao dịch trung bình và tuổi của card holder*

In [61]:
varAmt = is_frauddf.agg({'amt':'var_pop', 'Age':'var_pop'})
varAmt.show()

+------------------+------------------+
|      var_pop(amt)|      var_pop(Age)|
+------------------+------------------+
|15833.640752597597|278.34117269002166|
+------------------+------------------+



*Để tiện lợi hơn, ta có thể tính các giá trị cần biết của cột lượt giao dịch trung bình trong một bảng. Ở đây ta tính các giá trị tổng, trung bình, phương sai và độ lệch chuẩn chuẩn*

In [59]:
descriptiveAmt = is_frauddf.agg(sum("amt"), avg("amt"), var_pop('amt'), stddev_samp("amt"))
descriptiveAmt.show()

+-----------------+-----------------+------------------+------------------+
|         sum(amt)|         avg(amt)|      var_pop(amt)|  stddev_samp(amt)|
+-----------------+-----------------+------------------+------------------+
|92193.31000000003|69.16227306826708|15833.640752597597|125.87902077105512|
+-----------------+-----------------+------------------+------------------+



## **4.8. TÍNH COVARIANCE**

*Ta có thể tính covariance giữa cột lượt giao dịch trung bình và cột tuổi của card holder*

In [60]:
is_frauddf.cov('amt','Age')

84.93009110948925

## **4.9. TÍNH CORRELATION**

*Ta có thể tính correlation giữa cột lượt giao dịch trung bình và cột tuổi của card holder*

In [62]:
is_frauddf.corr('amt','Age')

0.04042560661652895

## **4.10. MÔ TẢ DATAFRAME**

Mô tả một DataFrame ứng dụng cho việc tính toán **thống kê đặc trưng** trên tất cả các cột trong một DataFrame. 

Trước tiên cần phải áp dụng hàm Des () lên từng cột của bảng

In [63]:
dataDescription = is_frauddf.describe()
dataDescription.show()

+-------+--------------------+-------------+------------------+--------------------+-----+------------------+-------------------+-----------------+--------------------+--------------------+-----------------+------------------+--------------------+------------------+
|summary|            merchant|     category|               amt|                city|state|               lat|               long|         city_pop|                 job|           trans_num|        merch_lat|        merch_long|            is_fraud|               Age|
+-------+--------------------+-------------+------------------+--------------------+-----+------------------+-------------------+-----------------+--------------------+--------------------+-----------------+------------------+--------------------+------------------+
|  count|                1333|         1333|              1333|                1333| 1333|              1333|               1333|             1333|                1333|                1333|          

Hàm Describe( ) sẽ trả về kết quả của các thống kê đặc trưng. Ở cột 1, kết quả trả về là tên của các thống kê đặc trưng như là: giá trị lớn nhất, nhỏ nhất, độ lệch chuẩn, giá trị trung bình, đếm. Ở cột 4 tên là "amt" cung cấp giá trị cho các thống kê đặc trưng ví dụ như giá trị 1.05 là số giá trị nhỏ nhất trong cột amt. 

Có thể chọn cột bất kỳ, ví dụ như cột city và city_pop trong bảng DataFrame qua hàm sau: 

In [64]:
dataDescriptioncitycitypop = is_frauddf.describe(['city', 'city_pop'])
dataDescriptioncitycitypop.show()

+-------+--------------------+-----------------+
|summary|                city|         city_pop|
+-------+--------------------+-----------------+
|  count|                1333|             1333|
|   mean|                null|96016.08552138034|
| stddev|                null|262532.5369151885|
|    min|         Albuquerque|               46|
|    max|Yellowstone Natio...|          1312922|
+-------+--------------------+-----------------+



Để có thể trả về những kết quả thống kê đặc trưng có chọn lọc hơn như là phương sai hoặc giá trị trung bình trên cột, ta không dùng hàm Describe( ) mà dùng hàm Summary( ) 

Hàm Summary( ) khá tương đồng với hàm Describe( ), tuy nhiên kết quả trả về có thêm điểm tứ phân vị (Quantiles) 25, 50, 75. 

In [68]:
summaryData = is_frauddf.summary()
summaryData.show()

+-------+--------------------+-------------+------------------+--------------------+-----+------------------+-------------------+-----------------+--------------------+--------------------+-----------------+------------------+--------------------+------------------+
|summary|            merchant|     category|               amt|                city|state|               lat|               long|         city_pop|                 job|           trans_num|        merch_lat|        merch_long|            is_fraud|               Age|
+-------+--------------------+-------------+------------------+--------------------+-----+------------------+-------------------+-----------------+--------------------+--------------------+-----------------+------------------+--------------------+------------------+
|  count|                1333|         1333|              1333|                1333| 1333|              1333|               1333|             1333|                1333|                1333|          

Cũng như hàm Describe( ), hàm Summary( ) cũng có thể chọn cột bất kỳ 

In [69]:
summarystatejob = is_frauddf.select('state','job'). summary('min','max')
summarystatejob.show()

+-------+-----+--------------------+
|summary|state|                 job|
+-------+-----+--------------------+
|    min|   AK|Administrator edu...|
|    max|   WY|  Wellsite geologist|
+-------+-----+--------------------+



## **4.11. SẮP XẾP DỮ LIỆU TRONG DATAFRAME**

- Theo thứ tự tăng dần


   Sắp xếp bảng dữ liệu theo ngày giao dịch sinh tăng dần(sớm nhất) : 

In [71]:
is_frauddfSorted1 = is_frauddf.orderBy("trans_date_trans_time")
is_frauddfSorted1.show(5)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac

- Theo thứ tự giảm dần

   Ngược lại với tăng dần ta có ngày giao dịch gần nhất: 

In [72]:
is_frauddfSorted2 =is_frauddf.orderBy("trans_date_trans_time", ascending=False)
is_frauddfSorted2.show(5)

+---------------------+--------------+--------------+-----+-----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|      merchant|      category|  amt|       city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------+--------------+-----+-----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-05 13:02:19|Dietrich-Fadel|health_fitness| 70.0|   Lakeport|   CA| 39.047|-122.9328|   11256|          Podiatrist|1972-10-18|726a1f0c5bc893784...|38.309802|-123.617622|       0| 50|
|  2019-01-05 12:42:03|    Barton LLC|     kids_pets|31.27|   Espanola|   NM|35.9866|-106.0654|   18408|Historic building...|1972-07-18|898f409ac52a44753...|35.178173|-106.125677|       0| 50|
|  2019-01-05 12:33:26|Erdman-Schad

- Sắp xếp 2 cột khác thứ tự

> Ví dụ như cột dân số tăng dần nhưng nghề nghiệp thì giảm dần(theo bảng chữ cái): 


In [73]:
is_frauddfSorted3 = is_frauddf.orderBy("city_pop","job", ascending=[False,True])
is_frauddfSorted3.show(5)

+---------------------+--------------------+-------------+------+-------+-----+-------+---------+--------+----------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|   city|state|    lat|     long|city_pop|       job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+-------+-----+-------+---------+--------+----------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-04 21:46:16|      Erdman-Schaden|personal_care| 86.09|Phoenix|   AZ|33.5623|-112.0559| 1312922|Contractor|1981-10-24|72227710ca3dd2a05...|33.954712|-111.883967|       0| 40|
|  2019-01-04 23:58:34|        Dickinson Lt|personal_care| 16.14|Phoenix|   AZ|33.5623|-112.0559| 1312922|Contractor|1981-10-24|242be021473092f80...|33.351328|-112.406605|       0| 40|
|  2019-01-05 03:06:15|         Hills-Olson|  grocery_net| 70.22|Phoenix|  

## **4.12 SẮP XẾP PHÂN VÙNG DỮ LIỆU**

-Chia dữ liệu thành 2 phân vùng: 

In [ ]:
is_frauddf = is_frauddf.repartition(2)
is_frauddf.rdd.glom().collect()

Sắp xếp ứng với 2 phân vùng đã phân

In [75]:
sortedPartitons = is_frauddf.sortWithinPartitions("city_pop","job", ascending=[False,True])
sortedPartitons.show()

+---------------------+--------------------+-------------+------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|     city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 02:32:26|          Spinka Inc|  grocery_net| 57.57|  Phoenix|   AZ|33.5623|-112.0559| 1312922|          Contractor|1981-10-24|0bffbd021d6322060...|32.635757|-112.195886|       0| 40|
|  2019-01-01 04:52:32|      O'Keefe-Hudson|  grocery_pos|209.89|  Phoenix|   AZ|33.5623|-112.0559| 1312922|          Contractor|1981-10-24|e6528b0d35f36cb29...|33.133991|-111.460639|       0| 40|
|  2019-01-01 1

Sử dụng câu lệnh này để thể hiện rõ từng giá trị thuộc phần nào 

In [ ]:
sortedPartitons.rdd.glom().collect()

## **4.13. XÓA CÁC DÒNG DỮ LIỆU TRÙNG LẶP**

**Việc xóa các dòng dữ liệu trùng lặp sẽ làm cho tệp dữ liệu được sạch hơn và giảm thiểu dung lượng lưu trữ.**

Để xóa các dòng trùng lặp, trước tiên ta tiến hành kiểm tra xem Dataset có các records bị trùng lặp hay không bằng lệnh count() và distinct.count()

In [4]:
is_frauddf.count()

1333

In [5]:
is_frauddf.distinct().count()

1333

Do cả 2 tệp dữ liệu đều có 1333 giá trị, như vậy, xem như tập dữ liệu không có các records bị trùng lặp. Tuy nhiên, ta có thể tự tạo ra các dòng trùng lặp như sau:

*Bước 1: Tạo ra một dòng dữ liệu random từ dataset*

In [6]:
import random
frauddf_list = is_frauddf.collect()
generation = random.choice(frauddf_list)

*Bước 2: Tạo ra một dataframe mới có kế thừa từ dataframe cũ nhưng có thêm dòng dữ liệu vừa chọn. Như vậy xem như dataframe mới có 1 dòng dữ liệu bị trùng lặp*

In [7]:
frauddf_list.append(generation)
header_list = is_frauddf.schema.names
dup_df = spark.createDataFrame(frauddf_list,header_list)
dup_df.show()

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac...|43.150704|-1

*Bước 3: Kiểm tra Dataframe mới bằng hàm count() và distinct.count()*

In [8]:
dup_df.count()

1334

In [9]:
dup_df.distinct().count()

1333

Có thể thấy, Dataframe mới đã có 1 dòng dữ liệu bị duplicate. Sau đó tiến hành xóa dòng dữ liệu duplicate như sau:

In [10]:
dup_df1 = dup_df.drop_duplicates()
dup_df1.show()

+---------------------+--------------------+-------------+------+------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|     category|   amt|              city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+-------------+------+------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 02:06:56|         Bradtke PLC|  grocery_pos| 78.13|         Louisiana|   MO|39.4336| -91.0664|    4593|Teacher early yea...|1940-11-11|e46a01d459665d346...|40.247129| -91.699005|       0|
|  2019-01-01 00:22:18|Nitzsche Kessler ...| shopping_pos|  4.02|         Valentine|   NE|42.8062|-100.6215|    4005|    Network engineer|1945-03-15|20490f3f0966ce74b...| 42.47559|-101.265846|

In [11]:
dup_df1.count()

1333

**Giả sử muốn xóa các dữ liệu bị duplicate khi xét theo từng cột, ta làm như sau**

In [12]:
dup_df2 = dup_df1.drop_duplicates(['category'])
dup_df2.show()

+---------------------+--------------------+--------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|      category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+--------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 00:00:51|      Lind-Buckridge| entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac...|43.150704|-112.154481|       0|
|  2019-01-01 07:13:01|       Jast and Sons|   food_dining| 74.09|              Corona|   CA|33.8419|-117.6043|  233717|  Wellsite geologist|1966-05-22|4a4cf450e1ee2741e...|33.9178

In [13]:
dup_df2.count()

14

Như vậy, do cột category chỉ có 14 dữ liệu riêng biệt, nên tệp dataframe sau khi xóa dữ liệu trùng lặp sẽ chỉ còn lại 14 dòng

**Tương tự, nếu muốn xóa các dòng dữ liệu riêng biệt khi xét trên tổ hợp 2 cột merchant và category, ta làm như sau:**

In [14]:
dup_df3 = dup_df1.drop_duplicates(['merchant','category'])
dup_df3.show()

+---------------------+--------------------+--------------+-------+------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|      category|    amt|              city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+--------------+-------+------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 14:43:44|       Abbott-Rogahn| entertainment|  74.63|           Ravenna|   NE|41.0233| -98.9041|    2202|  Solicitor Scotland|1974-06-21|dbe29230128b03874...|41.874726| -99.842215|       0|
|  2019-01-01 15:47:50|       Abbott-Steube| personal_care| 111.13|          Roseland|   NE|40.4591| -98.5551|     463| Nurse mental health|1948-05-31|40ca0f0111ef54d04...|40.300769| -

In [15]:
dup_df3.count()

568

Như vậy, tệp dữ liệu chỉ còn 568 dòng tổ hợp khác nhau của hai cột category và merchant.

## **4.14. TRÍCH SAMPLE TỪ DATASET**

**Thao tác này cho phép tạo ra một tệp dữ liệu mới là sample (mẫu thu nhỏ) của tệp dataset ban đầu, với các dữ liệu được lấy random. Điều này thuận tiện cho người dùng khi muốn trích xuất các dữ liệu ngẫu nhiên để thống kê mô tả.**

**Giả sử muốn trích một sample có kích cỡ bằng 20% tệp dataset ban đầu, với điều kiện các dòng dữ liệu không trùng lặp, ta làm như sau:**

In [25]:
fraud_sample = is_frauddf.sample(withReplacement=False,fraction=0.2,seed=200)
fraud_sample.show(5)

+---------------------+--------------------+------------+------+-----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|    category|   amt|       city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+------------+------+-----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 00:00:44|Heller Gutmann an...| grocery_pos|107.23|     Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0|
|  2019-01-01 00:46:18|Ferry Lynch and K...|    misc_net|  2.76|   San Jose|   CA|37.3304|-121.7913|  973849|      Science writer|1955-06-12|70ca7fe41f09770d7...|37.548452|-121.536454|       0|
|  2019-01-01 01:34:25|       

In [26]:
fraud_sample.count()

278

Kết quả thu được 293 dữ liệu ngẫu nhiên, chiếm 20% tệp dữ liệu ban đầu.

**Giả sử muốn trích một sample có kích cỡ bằng 20% tệp dataset ban đầu, với điều kiện các dòng dữ liệu có thể trùng lặp, ta làm như sau:**

In [23]:
fraud_sample1 = is_frauddf.sample(withReplacement=True,fraction=0.2,seed=200)
fraud_sample1.show(5)

+---------------------+--------------------+-------------+------+--------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|     category|   amt|          city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+-------------+------+--------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|        Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0|
|  2019-01-01 00:07:27|           Kiehn Inc|  grocery_pos| 96.29|       Grenada|   CA|41.6125|-122.5258|     589|     Systems analyst|1945-12-21|413636e759663f264...| 41.65752|-122.230347|       0|
|  2019-01

In [24]:
fraud_sample1.count()

281

**Giả sử muốn trích một sample với hệ quy chiếu là trường category, với 30% tệp được lấy từ hình thức thanh toán ăn tối (food dining), và 70% tệp được lấy từ hình thức thanh toán xăng dầu (gas transport), ta làm như sau:**

In [27]:
fraud_sample2 = is_frauddf.sampleBy(col='category',fractions={'food_dining':0.3,'gas_transport':0.7},seed=200)
fraud_sample2.show(5)

+---------------------+--------------------+-------------+------+--------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|     category|   amt|          city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+-------------+------+--------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 00:31:51| Ledner-Pfannerstill|gas_transport|102.13|      Thompson|   UT|38.9999| -109.615|      46|   Surveyor minerals|1987-04-23|47238da5b40d126c8...|39.807313|-109.348294|       0|
|  2019-01-01 00:40:50|     Cummerata-Jones|gas_transport| 70.53|        Athena|   OR|45.8289|-118.4971|    1302|              Dealer|1976-10-18|7b91bd200c11c5cd2...|46.043252|-117.962133|       0|
|  2019-01

In [28]:
fraud_sample2.count()

138

Như vậy, tệp sample có 145 dữ liệu, trong đó 30% tệp có hình thức thanh toán ăn tối (food dining), và 70% tệp có thanh toán xăng dầu (gas transport)

## **4.15. TÌM CÁC THÔNG TIN CÓ MẬT ĐỘ XUẤT HIỆN DÀY ĐẶC**

**Giả sử như muốn tìm xem các giao dịch được đến từ hình thức (category) nào nhiều nhất, ta làm như sau:**

In [29]:
is_frauddf.freqItems(cols=['category']).show()

+--------------------+
|  category_freqItems|
+--------------------+
|[food_dining, per...|
+--------------------+



Như vậy, có thể thấy, các giao dịch đến nhiều nhất từ việc thanh toán ăn tối (food dining)

**Giả sử như muốn tìm xem các giao dịch được đến từ hình thức (category) nào nhiều nhất, và được thực hiện bởi ai nhiều nhất, ta làm như sau:**

In [30]:
is_frauddf.freqItems(cols=['category','merchant']).show()

+--------------------+--------------------+
|  category_freqItems|  merchant_freqItems|
+--------------------+--------------------+
|[food_dining, per...|[Raynor Reinger a...|
+--------------------+--------------------+



Như vậy, có thể thấy, các giao dịch đến nhiều nhất từ việc thanh toán ăn tối (food dining), và người thực hiện nhiều nhất là "Raynor Reinger...".